In [1]:
import os
import json

In [2]:
data_folder = "mt-metrics-eval-v2/wmt23/sources/"
out_folder = "translations/wmt23/system-outputs"
ref_suffix = ".refA.txt"
ref_folder = "mt-metrics-eval-v2/wmt23/references/"
mapping = {
    "cs": "Czech",
    "uk": "Ukrainian",
    "de": "German",
    "en": "English",
    "he": "Hebrew",
    "ru": "Russian",
    "zh": "Chinese",
}

with open('mt_prompts.json', 'r') as file:
    prompts = json.load(file)

In [3]:
languages = ["de-en", "cs-uk", "en-zh"] #,
sample = 500

In [4]:
def load_sample(path, sample):
    loaded = []
    with open(path, 'r') as f:
        i = 0
        for line in f:
            if i >= sample:
                break
            loaded.append(line.strip())
            i += 1
    return loaded

In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM

os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3.1-8B-Instruct")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Meta-Llama-3.1-8B-Instruct", device_map="auto", quantization_config={"load_in_8bit": True})

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [6]:
from transformers import set_seed
set_seed(42)

In [7]:
LLAMA3_TEMPLATE = "<|begin_of_text|><|start_header_id|>system<|end_header_id|>" + \
"{{ system_prompt }}<|eot_id|><|start_header_id|>user<|end_header_id|>" + \
"{{ user_msg_1 }}<|eot_id|><|start_header_id|>assistant<|end_header_id|>"
# "{{ model_answer_1 }}<|eot_id|>"

# longer system prompt, came out of trying to make it shut up, not really needed with the chat template applied correctly.
SYSTEM_PROMPT = "You are a machine translation system. Rules:\n" + \
        "1. Provide a precise and correct translation in the target language.\n" + \
        "2. Give the answer without quotes, and without explanations.\n" + \
        "3. Don't say anything else after the translation.\n"
SYSTEM_PROMPT = "You are a helpful machine translation assistant."

In [ ]:
def run_inference(examples):
    encoded = tokenizer(examples, return_tensors="pt")
    encoded["input_ids"].to("cuda")
    prompt_length = encoded['input_ids'].shape[1]
            
    encoded["attention_mask"].to("cuda")
    output = model.generate(
        encoded["input_ids"].to("cuda"), 
        attention_mask=encoded["attention_mask"].to("cuda"), 
        max_new_tokens=200,
        num_beams=1,
        do_sample=False,
        temperature=None,
        top_p=None,
        #stop=[ "<|start_header_id|>", "<|end_header_id|>", "<|eot_id|>" ] 
    )
    output.to("cpu")
    translation = tokenizer.decode(output[0][prompt_length:], skip_special_tokens=True)
    print(translation)
    translations.append(translation.replace('\\n', ' ').replace('\\t', ' '))

In [ ]:
## the part I've actually been running
import pandas as pd
for lp in languages:
    # load testset
    testset = load_sample(f'{data_folder}/{lp}.txt', sample)
    references = load_sample(f'{ref_folder}{lp}{ref_suffix}', sample)
    
    for prompt in prompts:
        translations = []
        source_language = lp.split("-")[0]
        target_language = lp.split("-")[1]
        # Try the rest of the prompt variants
        if "src" not in prompt['id']:
            continue
        template = prompt['prompt'].replace("[target language]", mapping[target_language]).replace("[source language]", mapping[source_language])
        experiment_name = "rest_" + prompt['id']

        for item in testset:
            message = template.replace("[source sentence]", item) #.replace("### ", "")
            chat_template = LLAMA3_TEMPLATE.replace("{{ system_prompt }}", system_prompt).replace("{{ user_msg_1 }}", message)
            
            print("-----new item-----")
            print(chat_template)
            encoded = tokenizer(chat_template, return_tensors="pt")
            encoded["input_ids"].to("cuda")
            prompt_length = encoded['input_ids'].shape[1]
            
            encoded["attention_mask"].to("cuda")
            output = model.generate(
                encoded["input_ids"].to("cuda"), 
                attention_mask=encoded["attention_mask"].to("cuda"), 
                max_new_tokens=200,
                num_beams=1,
                do_sample=False,
                temperature=None,
                top_p=None,
                #stop=[ "<|start_header_id|>", "<|end_header_id|>", "<|eot_id|>" ] 
            )
            output.to("cpu")
            translation = tokenizer.decode(output[0][prompt_length:], skip_special_tokens=True)
            print(translation)
            translations.append(translation.replace('\\n', ' ').replace('\\t', ' '))
        
        # create folder translations/wmt23/system-outputs
        if not os.path.exists(f'translations/wmt23/system-outputs/{lp}'):
            os.makedirs(f'translations/wmt23/system-outputs/{lp}')
        dict = {'source': testset, 'translation': translations, 'reference': references}
        df = pd.DataFrame(dict)
        with open(f'translations/wmt23/system-outputs/{lp}/{experiment_name}.csv', 'w') as f:
            df.to_csv(f)


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


-----new item-----
<|begin_of_text|><|start_header_id|>system<|end_header_id|>You are a helpful machine translation assistant.<|eot_id|><|start_header_id|>user<|end_header_id|>### Translate this sentence from German to English, Source:\nDiego Cocca wird neuer Nationaltrainer von Mexiko\n### Target:<|eot_id|><|start_header_id|>assistant<|end_header_id|>


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.




Diego Cocca wird neuer Nationaltrainer von Mexiko
-----new item-----
<|begin_of_text|><|start_header_id|>system<|end_header_id|>You are a helpful machine translation assistant.<|eot_id|><|start_header_id|>user<|end_header_id|>### Translate this sentence from German to English, Source:\nIm Sommer 2026 richtet Mexiko gemeinsam mit den USA und Kanada die WM mit dann erstmals 48 Mannschaften aus. Cocca tritt die Nachfolge von Gerardo Martino an, dessen Vertrag nach dem frühen WM-Aus in Katar nicht verlängert wurde. Der 50-Jährige zählte bis vor wenigen Tagen nicht zu den Favoriten für den Posten. Er hatte erst im November als Coach des mexikanischen Clubs Tigres übernommen. Seine überraschende Berufung zum Nationaltrainer löste Ärger im Verein aus.\n### Target:<|eot_id|><|start_header_id|>assistant<|end_header_id|>


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...



Here is the translation of the text in English:

The credits per coin are set on the "Coin-Insertion Programming" display. High Score Display (High Score so far"). The points and initials of the player appear, including the next 10 scores. The values are used to set the high scores that were previously selected on the previous display (set to "back"). By pressing the "Up" switch, the initials QIX Standard are set. By pressing the "Test" switch, the next program display appears, "Total Evaluation". By pressing the "Up" switch, it can be set to 0. The following values are recorded: Total Credits: The number of paid credits. The total is not set to 0. When 1,000,000 is reached, the count starts over. The total can only appear as 0 when the battery or W1 connector is removed while the device is turned off. Left Coins: Number of coins inserted into the left coin
-----new item-----
<|begin_of_text|><|start_header_id|>system<|end_header_id|>You are a helpful machine translation assistant.<|

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.




Here is the translation of the given text from German to English:

High Score (Hit Rate): The number of players who had high enough hit rates to displace other players from the high score list. By pressing the "Up" switch, the values are reset to 0. By pressing the "Test" switch, the next program display appears.
-----new item-----
<|begin_of_text|><|start_header_id|>system<|end_header_id|>You are a helpful machine translation assistant.<|eot_id|><|start_header_id|>user<|end_header_id|>You are a machine translation system. Provide the English translation for this sentence in German: Einwurfschlitz-Programmierung/normale Geldeinstellung. Mit Hilfe dieser Anzeige kann von dem Aufsteller eine Standard-Münzeinstellung gewählt oder eine andere Einstellung programmiert werden\n<|eot_id|><|start_header_id|>assistant<|end_header_id|>


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.




Here is the translation of the given text in English:

Coin selection/normal coin setting. With the help of this display, the coin selector can be set to a standard coin setting or a different setting programmed.
-----new item-----
<|begin_of_text|><|start_header_id|>system<|end_header_id|>You are a helpful machine translation assistant.<|eot_id|><|start_header_id|>user<|end_header_id|>You are a machine translation system. Provide the English translation for this sentence in German: Ist eine Standardeinstellung vorgesehen, so wird durch Drücken des "Up"- und "Down"-Schalters die Programmierung gewählt, während die oberste Ziffer grün auf dem Bildschirm leuchtet. Die Größenwerte ändern sich mit der Änderung der Ziffer. Bei Erreichen der richtigen Einstellung durch Druck auf den "Test"-Schalter wird die Einstellung übernommen. Wird eine Einstellung gewünscht, die nicht der Standard-Einstellung entspricht, können die Werte durch Betätigen des "Sub-Test"-Schalters eingestellt werden. Nac

In [5]:
# This is the template that EuroLLM was trained with. Different models
# have different templates.
CHATML_TEMPLATE = """<|im_start|>system
You are a professional {source_language} to {target_language} translator. Your goal is to accurately convey the meaning and nuances of the original {source_language} text while adhering to {target_language} grammar, vocabulary, and cultural sensitivities.
<|im_end|>
<|im_start|>user
Translate the following {source_language} source text to {target_language}:
{source_language}: {source}
{target_language}: <|im_end|>
<|im_start|>assistant
"""


In [ ]:
### --- EuroLLM attempts --- 
import pandas as pd
from vllm import LLM, SamplingParams

sampling_params = SamplingParams(
    use_beam_search=False,
    best_of=1,
    # Since we are doing sentence level we can stop at newlines. This avoids the model trying to continue
    # generating translations
    stop=["<|im_end|>", "\n"], 
    min_p=0.02,
    temperature=1,
    max_tokens=4096,
)
os.environ["CUDA_VISIBLE_DEVICES"]="2"

llm = LLM(model="utter-project/EuroLLM-1.7B-Instruct", tensor_parallel_size=1, dtype="half")

# source_sentences = [s.strip() for s in open(f"data/sources/{args.lp}.txt").readlines()]

for lp in languages:
    # load testset
    testset = load_sample(f'{data_folder}/{lp}.txt', sample)
    references = load_sample(f'{ref_folder}{lp}{ref_suffix}', sample)
    experiment_name = "eurollm"
    model_inputs = []
    for source in testset:
        # Build MBR Inputs!
        source_language = mapping[args.lp.split("-")[0]]
        target_language = mapping[args.lp.split("-")[1]]

        source_prompt = CHATML_TEMPLATE.format(
            source_language=source_language,
            target_language=target_language,
            source=source,
        )
        # Each prompt will appear several times. VLLM also performs some prompt caching thus
        # this is actually fast.
        model_inputs.extend([source_prompt] * 1)  # from MBR tutorial so originally did * 20 or so

    print("Model inputs are built. Starting generate! This can take some time!")
    # Generate translations
    outputs = llm.generate(model_inputs, sampling_params)
    generations = [o.outputs[0].text for o in outputs]
    
    # create folder translations/wmt23/system-outputs
    if not os.path.exists(f'translations/wmt23/system-outputs/{lp}'):
        os.makedirs(f'translations/wmt23/system-outputs/{lp}')
    dict = {'source': testset, 'translation': generations, 'reference': references}
    df = pd.DataFrame(dict)
    with open(f'translations/wmt23/system-outputs/{lp}/{experiment_name}.csv', 'w') as f:
        df.to_csv(f)

In [9]:
message = "This is a test string."
tok = tokenizer(message, return_tensors="pt")
outputs = model.generate(inputs=tok["input_ids"].to("cuda"), attention_mask=tok["attention_mask"].to("cuda"))
outputs.to("cpu")
tokenizer.decode(outputs[0], skip_special_tokens=True)

{'input_ids': tensor([[128000,   2028,    374,    264,   1296,    925,     13]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]])}